# PHYS310 Final Project: Machine Learning Interaction Potentials
## For runtime reasons, this notebook loads a pre-trained model.
The later cross validation demonstration is done with significantly fewer epochs than this model was trained for, for brevity.

In [ ]:
import model
import keras

import tensorflow as tf
import crystal_loader

import numpy as np
from symmetry import *
import dill
from sklearn.preprocessing import StandardScaler
import h5py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import os
from tqdm import tqdm
import MLPtools

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

## Loading features

Generating all of the features takes a couple minutes to run even when running parallel across multiple cores, so I've serialized the pre-regularization BP features in a more readily-accessible h5 file. Feature generation code can be found in `symmetry.py` and `symmetry.py`, and the final execution is done in the notebook `feature_generation.ipynb`. There are two different training label sets prepared, one with raw DFT total energies and another with the computed cohesive energies. Training was done with the latter to follow the `aenet` paper and compare data on an equivalent basis.

In [ ]:
dset_name = "TiO2_2015_angfixed_x3"

features_path = f"../pickles/{dset_name}_features.h5"
labels_path = f"../pickles/{dset_name}_labeldata.h5"


with h5py.File(features_path, "r") as f:
    features = [f[f"array_{i}"][:] for i in range(len(f))]

label_df = pd.read_hdf(labels_path, key="labels")
n_atoms = pd.read_hdf(labels_path, key="n_atoms").to_numpy().reshape(-1, 1)

print(label_df.columns)

In [ ]:
# select label to use
labels = label_df["cohesive_energy"].to_numpy().reshape(-1, 1)


## Standardization
All features are regularized through a StandardScaler prior to any training done. Data is compiled into a set of RaggedTensors.

The commented-out code is copied from an initial attempt to address the dataset's unbalanced nature by attempting to weigh the loss of rarer instances with larger structures where the model was observed to lose accuracy. This resulted in poorer training results so I stopped using it.

In [ ]:
def scale_ragged(features):
    stacked = np.vstack(features)
    SSC = StandardScaler().fit(stacked)
    scaled_features = [SSC.transform(struct) for struct in features]

    return scaled_features

scaled_features = scale_ragged(features)

Xtrain, Xtest, y_train, y_test, c_train, c_test = train_test_split(scaled_features, labels, n_atoms, shuffle=True, random_state=12, test_size=0.2)
Xval, Xtest, y_val, y_test, c_val, c_test = train_test_split(Xtest, y_test, c_test, shuffle=True, random_state=12, test_size=0.5)

Xtrain = tf.ragged.constant(Xtrain, ragged_rank=1, inner_shape=(70,))
Xval = tf.ragged.constant(Xval, ragged_rank=1, inner_shape=(70,))
Xtest = tf.ragged.constant(Xtest, ragged_rank=1, inner_shape=(70,))

# from sklearn.neighbors import KernelDensity

# def inverse_density_weights(y, **kwargs):
#     kde = KernelDensity(kernel='gaussian', bandwidth=5.0).fit(y.reshape(-1, 1))

#     # 2. Estimate density
#     log_density = kde.score_samples(y.reshape(-1, 1))
#     density = np.exp(log_density)

#     # 3. Invert density for weights
#     weights = 1 / (density + 1e-6)
#     weights /= np.mean(weights)  # optional normalization

#     return weights.reshape(-1, 1)

# y_train_weights = inverse_density_weights(y_train)
# y_val_weights = inverse_density_weights(y_val)

## Subnet and Model Architecture
The final subnet architecture is the same one used in the paper (70-10-10-1). A major difference between the two however is that we train with ADAM here rather than L-BFGS. I couldn't find a straightforward way of using the latter with `keras` models and `fit()`.

In [ ]:
layers = [keras.layers.Dense(10, activation="relu"),
          keras.layers.Dense(10, activation="relu")]

MLP1 = model.MLPNet(layers=layers, N_features=70, ragged_processing=False)
MLP1.built = True

In [ ]:
# open a previous  model's training history
modname = "x3v2_weighted"
with open(f'./saved_models/{modname}_history.dill', 'rb') as file_pi:
    epochs, hist = dill.load(file_pi)

## Training History
Losses are reported in the metric the model was trained in (MSE eV). Because of limitations with Keras.Metric objects and a lack of time, I was not able to implement per-atom losses as a usable validation metric during training, but they can still be calculated externally after training.

In [ ]:
# load a previous model's weights
loaded_model_name = modname
mpath = f'./saved_models/{loaded_model_name}.weights.h5'
print(os.getcwd())
MLP1.load_weights(mpath)

In [ ]:
import matplotlib.pyplot as plt

plt.close(1)
fig, ax = plt.subplots(1, 2, num=1, figsize=(10, 5), tight_layout=True)
ax[0].plot(epochs, hist["loss"], label="training")
ax[0].plot(epochs, hist["val_loss"], label="validation")

ax[0].set_ylabel("loss (MSE)", fontsize=16)

ax[1].plot(epochs, hist["root_mean_squared_error"], label="_training")
ax[1].plot(epochs, hist["val_root_mean_squared_error"], label="_validation")
# ax[1].set_yscale("log")
ax[0].set_yscale("log")
ax[1].set_ylabel("loss (RMSE), eV", fontsize=16)

axmini = ax[1].inset_axes(bounds=[0.3, 0.4, 0.67, 0.58])
axmini.tick_params(axis='both', which='major', labelsize=10)
axmini.plot(epochs[1150:], hist["root_mean_squared_error"][1150:])
axmini.plot(epochs[1150:], hist["val_root_mean_squared_error"][1150:])



ax[0].set_xlabel("epoch", fontsize=16)
ax[1].set_xlabel("epoch", fontsize=16)


fig.legend(loc=(0.8, .25), fontsize=14)

## Trained Model Evaluation

The project comes with a few included helper libaries to calculate useful metrics like atomic energy contributions and losses.

In [ ]:
X_exam_tr = Xtrain
y_pred_tr = np.squeeze(MLP1.predict(X_exam_tr))
y_exam_tr = np.squeeze(y_train)
target_counts_tr = np.squeeze(c_train)

y_pred_atomic_tr = MLPtools.atomic_energies(y_pred_tr, target_counts_tr)
y_exam_atomic_tr = MLPtools.atomic_energies(y_exam_tr, target_counts_tr)

print(fr"train loss RMSE: {np.sqrt(MLPtools.atomic_MSE(y_exam_tr, y_pred_tr, target_counts_tr)) * 1000} meV/atom")



X_exam = Xtest
y_pred = np.squeeze(MLP1.predict(X_exam))
y_exam = np.squeeze(y_test)
target_counts = np.squeeze(c_test)

y_pred_atomic = MLPtools.atomic_energies(y_pred, target_counts)
y_exam_atomic = MLPtools.atomic_energies(y_exam, target_counts)

print(fr"test loss RMSE: {np.sqrt(MLPtools.atomic_MSE(y_exam, y_pred, target_counts)) * 1000} meV/atom")

## Data Categorization
As the structures in the dataset fall into a few rough size classes, it makes sense to analyze the model's performance across these different classes and look for discrepancies.

In [ ]:
fig, ax = plt.subplots(num=124)
ax.hist(n_atoms, bins="auto")
ax.set_xlabel("structure size (atoms)")
plt.show()

In [ ]:
# roughly bin atomic size classes
size_classes = np.unique(np.floor(5 * np.log(np.unique(target_counts))))

filters = []
for size_class in size_classes:
    class_filt = np.floor(5 * np.log(target_counts)) == size_class
    filters.append(class_filt.flatten())


size_classes_tr = np.unique(np.floor(5 * np.log(np.unique(target_counts_tr))))

filters_tr = []
for size_class in size_classes_tr:
    class_filt = np.floor(5 * np.log(target_counts_tr)) == size_class
    filters_tr.append(class_filt.flatten())   

In [ ]:
fig, ax = plt.subplots(2, 4, num=2, figsize=(20, 10), tight_layout=True)
matplotlib.rcParams.update({'font.size': 20})

for num, (axes, class_filt, size_class) in enumerate(zip(ax[0,:], filters, size_classes)):
    pred_species = y_pred[class_filt]
    exam_species = y_exam[class_filt]
    axes.set_title(fr"$N_{{atoms}} \sim {np.round(np.exp(size_class / 5)):.0f}$")
    # axes.text(0.55, 0.5, f"{np.mean(class_filt * 100):.2f}%\nof instances", transform=axes.transAxes)
    axes.set_xlabel("Cohesive energy (eV)", fontsize=18)
    axes.hist(exam_species, bins="auto", label="true" if num == 0 else None, alpha=0.6)
    axes.hist(pred_species, bins="auto", label="predicted" if num == 0 else None, alpha=0.7)
    axes.tick_params(axis='both', which='major', labelsize=18)
    if num in [0, 1]:
        axmini = axes.inset_axes(bounds=(0.3, 0.2, 0.7, 0.25))
        axmini.tick_params(axis='both', which='major', labelsize=0)
        par_lims = axes.get_xlim()
        n = axmini.hist(exam_species, bins="auto", label="true" if num == 0 else None, alpha=0.6)
        axmini.hist(pred_species, bins="auto", label="predicted" if num == 0 else None, alpha=0.7)

        
        width = par_lims[1] - par_lims[0]
        axmini.set_xlim(par_lims[0] + width * 0.3, par_lims[1])
        axmini.set_ylim(0, np.max(n[0][len(n[0]) // 2:]) *1.3)

    fig.legend(loc=(0.05, 0.8), fontsize=24)

for num, (axes, class_filt, size_class) in enumerate(zip(ax[1,:], filters, size_classes)):
    pred_species = y_pred_atomic[class_filt]
    exam_species = y_exam_atomic[class_filt]
    axes.set_xlabel("Cohesive energy per atom (eV/atom)", fontsize=16)
    axes.hist(exam_species, bins="auto", label="true", alpha=0.6)
    axes.hist(pred_species, bins="auto", label="predicted", alpha=0.7)
    if num in [0, 1]:
        axmini = axes.inset_axes(bounds=(0.3, 0.2, 0.7, 0.25))
        axmini.tick_params(axis='both', which='major', labelsize=0)
        par_lims = axes.get_xlim()
        n = axmini.hist(exam_species, bins="auto", label="true" if num == 0 else None, alpha=0.6)
        axmini.hist(pred_species, bins="auto", label="predicted" if num == 0 else None, alpha=0.7)

        
        width = par_lims[1] - par_lims[0]
        axmini.set_xlim(par_lims[0] + width * 0.3, par_lims[1])
        axmini.set_ylim(0, np.max(n[0][len(n[0]) // 2:]) *1.3)

    # axes.legend()

Above figure: histogram of model predictions for the test set. Note the lack of model accuracy on large structures. There appears to be a correlation between model fit quality and the amount of training data supplied for a given size class.

In [ ]:
fig, ax = plt.subplots(num=3, figsize=(10, 7.5), tight_layout=True)

ax.scatter(y_exam_atomic, y_pred_atomic, marker="o", edgecolor="k")
ax.set_title("Prediction vs truth plot (cohesive energy/atom)")
axmin = np.min([np.min(y_exam_atomic), np.min(y_pred_atomic)])
axmax = np.max([np.max(y_exam_atomic), np.max(y_pred_atomic)])
x = np.linspace(axmin, axmax)
ax.plot(x, x, c="r", linestyle="--", lw=0.5)

fig, ax = plt.subplots(2, 4, num=2, figsize=(20, 10), tight_layout=True)
# fig.suptitle("Predicted vs Actual plots across different size classes")

textpos = (0.05, 0.85)
errors = []
errors_tr = []

for num, (axes, class_filt, tr_filt, size_class) in enumerate(zip(ax[0,:], filters, filters_tr, size_classes)):
    axes.set_title(fr"($N_{{atoms}} \sim {np.round(np.exp(size_class / 5)):.0f}$)",
                  pad=15)
    pred_species = y_pred_atomic[class_filt]
    exam_species = y_exam_atomic[class_filt]
    species_counts = target_counts[class_filt]

    pred_species_tr = y_pred_atomic_tr[tr_filt]
    exam_species_tr = y_exam_atomic_tr[tr_filt]
    species_counts_tr = target_counts_tr[tr_filt]

    errors.append(np.sqrt(keras.metrics.MeanSquaredError()(exam_species, pred_species)) * 1000)


    axes.text(*textpos, f"$N_{{instances}}={np.sum(class_filt):.0f}$", transform=axes.transAxes)
    
    # axes.set_title(fr"Energy per atom")
    axes.set_aspect("equal")
    axes.set_aspect("equal")

    axes.scatter(exam_species, pred_species, lw=0, marker=".", s=50, c=np.arange(np.size(exam_species)), cmap="viridis")
    axmin = np.min([np.min(exam_species_tr), np.min(pred_species_tr)])
    axmax = np.max([np.max(exam_species_tr), np.max(pred_species_tr)])
    x = np.linspace(axmin, axmax)
    axes.plot(x, x, c="r", linestyle="--", lw=0.5)
    axes.set_xlim(axmin, axmax)
    axes.set_ylim(axmin, axmax)


for num, (axes, class_filt, tr_filt, size_class) in enumerate(zip(ax[1,:], filters, filters_tr, size_classes)):
    pred_species_tr = y_pred_atomic_tr[tr_filt]
    exam_species_tr = y_exam_atomic_tr[tr_filt]
    species_counts_tr = target_counts_tr[tr_filt]

    axes.text(*textpos, f"$N_{{instances}}={np.sum(tr_filt):.0f}$", transform=axes.transAxes)
    
    axes.set_aspect("equal")
    axes.set_aspect("equal")
    errors_tr.append(np.sqrt(keras.metrics.MeanSquaredError()(exam_species_tr, pred_species_tr)) * 1000)
    axes.scatter(exam_species_tr, pred_species_tr, lw=0, marker=".", s=50, c=np.arange(np.size(exam_species_tr)), cmap="magma")

    axmin = np.min([np.min(exam_species_tr), np.min(pred_species_tr)])
    axmax = np.max([np.max(exam_species_tr), np.max(pred_species_tr)])
    x = np.linspace(axmin, axmax)
    axes.plot(x, x, c="r", linestyle="--", lw=0.5)
    axmin = -9
    axmax = -4
    axes.set_xlim(axmin, axmax)
    axes.set_ylim(axmin, axmax)

fig.supylabel("Predicted energy (eV/atom)", fontsize=24)
fig.supxlabel("True energy (eV/atom)", fontsize=24)


# Model Validation
We procedurally generate or modify some test structures to see if the model makes physically reasonable predictions about them.
This section also serves as an opportunity to demonstrate the structure-to-feature pipeline (the ones used for training have long been pre-computed).

In [ ]:
import pandas as pd
import crystal_loader
import symmetry


Rc = 6.5 # A
Rs = 0

# Behler-Parinello symmetry function parameters, specified for every type of interaction.

# build radial basis params:

radial_etas = [0.003214, 0.035711, 0.071421, 0.124987, 0.214264, 0.357106, 0.714213, 1.428426]
params_radial = tuple([{"Rs": Rs, "eta": eta} for eta in radial_etas])

angular_etas = [0.000357, 0.028569, 0.089277, 0.000357, 0.028569, 0.089277, 0.000357, 0.028569, 0.089277] * 2
angular_lambdas = [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
angular_zetas = [1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0]

params_angular = tuple([{"Rs": Rs, "eta": eta, "lmbda": lam, "zeta": zeta} for eta, lam, zeta in zip(angular_etas, angular_lambdas, angular_zetas)])

tio_params_ang = {("Ti", "Ti"): params_angular, ("Ti", "O"): params_angular, ("O", "O"): params_angular}
tio_params_rad = {"Ti": params_radial, "O": params_radial}

## Test 1: Scaling the lattice parameters of a pre-existing structures

The lattice vectors and cartesian coordinates of $N=24$ atom structures from the dataset are scaled from 0.5x to 2x, simulating a corresponding physical scaling of the entire structure.

In [ ]:
from MLPtools import scale_ragged, moving_average

scale_factor = np.logspace(-1, 1, 2000, base=2)
scales = scale_factor.reshape(-1, 1)

def scale_structure(struct, LVs, scale):
    scaled_coords = MLPtools.get_coordinates(struct) * scale
    scaled_LVs = LVs * scale
    scaled_struct = struct.copy()
    scaled_struct.update(scaled_coords)
    return scaled_struct, scaled_LVs


def get_energies(fpath):
    # load structures
    structs, labels, lattice_vectors, counts = crystal_loader.get_dataset(fpath, halt=5)
    
    # calculate cohesive energies using DFT total energies and individual atom energies
    E_O = -432.503149303
    E_Ti = -1604.604515075
    
    E_leg = {"Ti": E_Ti, "O": E_O}
    
    cohesive_e = crystal_loader.get_cohesive_energies(labels, counts, E_leg)
    n_atoms = counts.sum(axis=1)
    e_per_atom = np.squeeze(cohesive_e) / n_atoms

    # scale structures
    
    structure_series, LV_series = MLPtools.transform_structures(structs, lattice_vectors, transformation=scale_structure, variables=scales)
    
    # build features
    
    shrinking_series = []
    
    for structs, LVs in zip(structure_series, LV_series):
        shrinking_features = crystal_loader.build_features(structs, LVs, Rc=6.5, params_rad=tio_params_rad, params_ang=tio_params_ang)
        shrinking_series.append(shrinking_features)
    
    # predict new energies
    shrinking_energy_series = []
    print("Predicting new energies")
    for series, atom_count, in tqdm(zip(shrinking_series, n_atoms)):
        # standard scale off of original dataset
    
        scaled_features_shr = scale_ragged(series, fit_target=features)
    
        Xshrink = tf.ragged.constant(scaled_features_shr, ragged_rank=1, inner_shape=(70,))
        y_pred = MLP1.predict(Xshrink, verbose=0, batch_size=1)
        shrinking_energy_series.append(1000 * y_pred / atom_count)

    return shrinking_energy_series, e_per_atom

In [ ]:
shrinking_energy_series_20, e_per_atom_20 = get_energies("../data/TiO2_N~20")

## Raw output for a single structure:

In [ ]:
%matplotlib widget

plt.close(15)
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
fig, axz = plt.subplots(num=15, tight_layout=True, figsize=(10, 6))
ax = axz
axmini = ax.inset_axes((0.5, 0.3, 0.48, 0.68))
axmini.tick_params(axis='both', which='major', labelsize=10)
par_lims = axes.get_xlim()
for num, (y_pred, y_true) in enumerate(zip(shrinking_energy_series_20[:], e_per_atom_20)):
    ax.scatter(scale_factor, y_pred, lw=0.1, marker=".", c=np.arange(np.size(y_pred)), cmap="viridis")

    axmini.scatter(scale_factor, y_pred, lw=0.1, marker=".", c=np.arange(np.size(y_pred)), cmap="viridis")
    
    width = par_lims[1] - par_lims[0]
    axmini.set_xlim(0.8, 1.1)
    axmini.set_ylim(-9000, -5000)

    
    break

ax.set_xlabel(r"Scale Factor $\frac{a}{a_0}$")
ax.set_ylabel('Predicted Energy (meV / atom)')
ax.axhline(0, 0, 1, linestyle="--", color="tab:green")
ax.set_xlim(0.5, 2)
plt.show()


## Running a moving-average window to estimate smooth-curve behavior

In [ ]:
plt.close(15)
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
fig, axz = plt.subplots(num=15, tight_layout=True, figsize=(10, 6))
ax = axz
axmini = ax.inset_axes((0.5, 0.3, 0.48, 0.68))
axmini.tick_params(axis='both', which='major', labelsize=10)
par_lims = axes.get_xlim()
for num, (y_pred, y_true) in enumerate(zip(shrinking_energy_series_20[:], e_per_atom_20)):
    color = color_cycle[num % len(color_cycle)]
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    minimum = np.min(y)
    minimum_x = x[np.nonzero(y == minimum)[0][0]]
    ax.plot(x, y, label=num, alpha=1, color=color)
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    ax.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    
    # predict actual structure energy
    axmini.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    axmini.plot(x, y, label=num, alpha=1, color=color)

    axmini.errorbar(1, y_true * 1000, yerr=0, capsize=0, elinewidth=0, markersize=8, marker="*", color=color, mec="k")
    axmini.scatter(minimum_x, minimum, marker="x", color=color, s=48)
    print(minimum_x, minimum)

    width = par_lims[1] - par_lims[0]
    axmini.set_xlim(0.95, 1.05)
    axmini.set_ylim(-9000, -7500)



ax.set_xlabel(r"Scale Factor $\frac{a}{a_0}$")
ax.set_ylabel('Predicted Energy (meV / atom)')
ax.axhline(0, 0, 1, linestyle="--", color="tab:green")
ax.set_xlim(0.5, 2)
ax.set_title("N~20")
plt.show()


## N$\sim$81 (this one may take a while to run)

In [ ]:
shrinking_energy_series_81, e_per_atom_81 = get_energies("../data/TiO2_N~81")

In [ ]:
plt.close(15)
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
fig, ax = plt.subplots(num=15, tight_layout=True, figsize=(10, 6))
axmini = ax.inset_axes((0.52, 0.06, 0.48, 0.65))
axmini.tick_params(axis='both', which='major', labelsize=10)
par_lims = axes.get_xlim()
for num, (y_pred, y_true) in enumerate(zip(shrinking_energy_series_81[:], e_per_atom_81)):
    color = color_cycle[num % len(color_cycle)]
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    minimum = np.min(y)
    minimum_x = x[np.nonzero(y == minimum)[0][0]]
    ax.plot(x, y, label=num, alpha=1, color=color)
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    ax.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    
    # predict actual structure energy
    axmini.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    axmini.plot(x, y, label=num, alpha=1, color=color)

    axmini.errorbar(1, y_true * 1000, yerr=0, capsize=0, elinewidth=0, markersize=8, marker="*", color=color, mec="k")
    axmini.scatter(minimum_x, minimum, marker="x", color=color, s=48)
    print(minimum_x, minimum)

    width = par_lims[1] - par_lims[0]
    axmini.set_xlim(0.90, 1.1)
    axmini.set_ylim(-9600, -6000)


ax.set_xlabel(r"Scale Factor $\frac{a}{a_0}$")
ax.set_ylabel('Predicted Energy (meV / atom)')
ax.set_title("N~81")
ax.axhline(0, 0, 1, linestyle="--", color="tab:green")
ax.set_xlim(0.5, 2)
plt.show()

## N~45

In [ ]:
shrinking_energy_series_45, e_per_atom_45 = get_energies("../data/TiO2_N~45")

In [ ]:
plt.close(15)
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
fig, ax = plt.subplots(num=15, tight_layout=True, figsize=(10, 6))
axmini = ax.inset_axes((0.5, 0.35, 0.48, 0.63))
axmini.tick_params(axis='both', which='major', labelsize=10)
par_lims = axes.get_xlim()
for num, (y_pred, y_true) in enumerate(zip(shrinking_energy_series_45[:], e_per_atom_45)):
    color = color_cycle[num % len(color_cycle)]
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    minimum = np.min(y)
    minimum_x = x[np.nonzero(y == minimum)[0][0]]
    ax.plot(x, y, label=num, alpha=1, color=color)
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    ax.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    
    # predict actual structure energy
    axmini.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    axmini.plot(x, y, label=num, alpha=1, color=color)

    axmini.errorbar(1, y_true * 1000, yerr=0, capsize=0, elinewidth=0, markersize=8, marker="*", color=color, mec="k")
    axmini.scatter(minimum_x, minimum, marker="x", color=color, s=48)
    print(minimum_x, minimum)

    width = par_lims[1] - par_lims[0]
    axmini.set_xlim(0.95, 1.05)
    axmini.set_ylim(-9000, -7200)


# ax.set_xlabel(r"Scale Factor $\frac{a}{a_0}$")
ax.set_ylabel('$\overline{\epsilon}_{pred}$ (meV / atom)')
ax.set_title("N~45")
ax.axhline(0, 0, 1, linestyle="--", color="tab:green")
ax.set_xlim(0.5, 2)
plt.show()

## N~5

In [ ]:
shrinking_energy_series_5, e_per_atom_5 = get_energies("../data/TiO2_N~5")

In [ ]:
plt.close(15)
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
fig, ax = plt.subplots(num=15, tight_layout=True, figsize=(10, 6))
axmini = ax.inset_axes((0.5, 0.3, 0.48, 0.68))
axmini.tick_params(axis='both', which='major', labelsize=10)
par_lims = axes.get_xlim()
for num, (y_pred, y_true) in enumerate(zip(shrinking_energy_series_5[:], e_per_atom_5)):
    color = color_cycle[num % len(color_cycle)]
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    minimum = np.min(y)
    minimum_x = x[np.nonzero(y == minimum)[0][0]]
    ax.plot(x, y, label=num, alpha=1, color=color)
    x, y, std = moving_average(scale_factor, y_pred, window=51)
    ax.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    
    # predict actual structure energy
    axmini.fill_between(x, y - std, y + std, alpha=0.2, color=color)
    axmini.plot(x, y, label=num, alpha=1, color=color)

    axmini.errorbar(1, y_true * 1000, yerr=0, capsize=0, elinewidth=0, markersize=8, marker="*", color=color, mec="k")
    axmini.scatter(minimum_x, minimum, marker="x", color=color, s=48)
    print(minimum_x, minimum)

    width = par_lims[1] - par_lims[0]
    axmini.set_xlim(0.95, 1.05)
    axmini.set_ylim(-9000, -7500)


# ax.set_xlabel(r"Scale Factor $\frac{a}{a_0}$")
ax.set_title("N~5")
# ax.set_ylabel('Predicted Energy (meV / atom)')
ax.set_ylabel('$\overline{\epsilon}_{pred}$ (meV / atom)')
ax.axhline(0, 0, 1, linestyle="--", color="tab:green")
ax.set_xlim(0.5, 2)
plt.show()